In [1]:
import tensorflow as tf
import numpy as np

/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/luokui/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/luokui/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# tf test

In [ ]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=2))

x = tf.constant([[0.7, 0.1]])
a = tf.matmul(x, w1)
b = tf.matmul(a, w2)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) # 注意这里的写法
    print(sess.run(b))

# test TF2

In [80]:
# 构造一个简单的数据集。
train_x = np.r_[np.random.randn(10000, 2), np.random.randn(10000, 2) + 2]
train_y = np.r_[np.repeat(-1, 10000), np.repeat(1, 10000)].reshape(-1, 1)

def generate_data(x, y, size=50):
    alls = y.shape[0]
    n = int(alls / size)
    for i in range(n-1):
        yield x[size * i : size * (i+1)] , y[size * i : size * (i+1)]

In [83]:
w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=2))
x = tf.placeholder(tf.float32, shape=[None, 2], name="input_x")
y_ = tf.placeholder(tf.float32, shape=[None, 1], name="out_y")
model_1 = tf.matmul(x, w1)
model_2 = tf.matmul(model_1, w2)
loss = - tf.reduce_mean(-y_ * tf.log(tf.clip_by_value(model_2, 1e-10, 1)))
train_op = tf.train.AdamOptimizer(0.001).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):
        for d_x, d_y in generate_data(train_x, train_y):
            sess.run(train_op, feed_dict={x: d_x, y_: d_y})
        if i % 5==0:
            losses = sess.run(loss, feed_dict={x:train_x, y_: train_y})
            print("losses is {}".format(losses))
    print(sess.run(w1))
    print(sess.run(w1))

losses is 5.7107648849487305
losses is 5.575189113616943
[[-0.6642167   1.6406729  -0.1005234 ]
 [-2.7850468  -0.27976224  0.9921207 ]]
[[-0.6642167   1.6406729  -0.1005234 ]
 [-2.7850468  -0.27976224  0.9921207 ]]


# Stack denoisingEncoder

In [2]:
train_x = np.r_[np.random.randn(10000, 5), np.random.randn(10000, 5) + 2]

def gen_data(x, size=50):
    alls = x.shape[0]
    n = int(alls / size)
    for i in range(n-1):
        yield x[size * i : size * (i+1)]

class denoise_auto:
    def __init__(self, hidden_size, input_shape):
        self.hidden_size = hidden_size
        self.input_shape = input_shape
        
        self.x = tf.placeholder(tf.float32, shape=[None, self.input_shape], name="input_x")
        self.mask = tf.placeholder(tf.float32, shape=[None, self.input_shape], name="mask_s")
        
        w_init_max = 4 * np.sqrt(6./ (self.hidden_size + self.input_shape))
        self.w1 = tf.Variable(tf.random_uniform(shape=[self.input_shape, self.hidden_size], minval=-w_init_max,
                                  maxval=w_init_max), name="input_w1")
        self.b1 = tf.Variable(tf.zeros([self.hidden_size]), name="b1")
        self._w1_t = tf.transpose(self.w1)
        self._b1_t = tf.Variable(tf.zeros([self.input_shape]), name="b1_t")
        
        tild_x = tf.multiply(self.x, self.mask)
        # 这里有主要 如何 使用masking
        self.y = tf.nn.relu(tf.matmul(tild_x, self.w1) + self.b1)
        z = tf.nn.sigmoid(tf.matmul(self.y, self._w1_t) + self._b1_t)
        self.cost = tf.reduce_mean(tf.pow(z - self.x, 2))
        self.train_op = tf.train.GradientDescentOptimizer(0.01).minimize(self.cost)
        
        
def fitdatas(train_x, hidden_size, mask_ratio=0.8, epochs=50, batch_size=200):
    input_shape = train_x.shape[1]
    with tf.Session() as sess:
        model = denoise_auto(hidden_size=hidden_size, input_shape=input_shape)
        sess.run(tf.global_variables_initializer())
        for i in range(epochs):
            for d_x in gen_data(train_x, size=batch_size):
                mask_vale = np.random.binomial(1, mask_ratio, size=d_x.shape)
                feed_dicts = {model.x: d_x, model.mask: mask_vale}
                _, losses = sess.run([model.train_op, model.cost], feed_dict=feed_dicts)
            if i % 10 == 0:
                print("iter{} , losses is {}".format(i, losses))      
        mask_encoder = np.random.binomial(1, 1, size=train_x.shape)
        encoder_x, w1, b1 = sess.run([model.y, model.w1, model.b1], feed_dict={model.x: train_x, model.mask: mask_encoder})
    return w1, b1, encoder_x
    
#w_1, b_1, encoders = fitdatas(train_x, hidden_size=10)

iter0 , losses is 2.455381393432617
iter10 , losses is 2.09230375289917
iter20 , losses is 2.0386877059936523
iter30 , losses is 2.0552096366882324
iter40 , losses is 2.0176775455474854


In [58]:
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
# 构造一个简单的数据集。
train_x = np.r_[np.random.randn(10000, 5), np.random.randn(10000, 5) + 2, np.random.randn(10000, 5) + 3]
train_y = np.r_[np.repeat(0, 10000), np.repeat(1, 10000), np.repeat(2, 10000)].reshape(-1, 1)

from sklearn.preprocessing import OneHotEncoder
# binarizer 也是一个有用的地方。
#ohot = OneHotEncoder()
#train_y_onehot = ohot.fit_transform(train_y).toarray()
train_y_onehot = OneHotEncoder().fit_transform(train_y).toarray()

def generate_data(x, y, size=100):
    alls = y.shape[0]
    n = int(alls / size)
    for i in range(n-1):
        yield x[size * i : size * (i+1)] , y[size * i : size * (i+1)]

In [60]:
lists = [10, 10, 10]
auto_outs = []
w_outs = []
b_outs = []
fit_data = train_x
for index, size in enumerate(lists):
    w, b, encoders =  fitdatas(fit_data, hidden_size=size, epochs=10)
    auto_outs.append(encoders)
    w_outs.append(w)
    b_outs.append(b)
    fit_data = encoders

iter0 , losses is 5.606483459472656
iter0 , losses is 84.56307983398438
iter0 , losses is 895.0272216796875


In [93]:
input_shape = 5
y_shape = 3

x_stack = tf.placeholder(tf.float32, shape=[None, input_shape], name="stack_x")
y_stack = tf.placeholder(tf.float32, shape=[None, y_shape], name="stack_y")

fit_stack = x_stack
for index in range(len(lists)):
    temp_w = tf.Variable(w_outs[index], name="lay_w{}".format(index))
    temp_b = tf.Variable(b_outs[index], name="lay_b{}".format(index))
    #temp_input = tf.
    fit_stack = tf.nn.sigmoid(tf.nn.xw_plus_b(fit_stack, temp_w, temp_b),
                              name="encod_{}".format(index))
    

last_w = tf.Variable(tf.truncated_normal([fit_stack.get_shape()[1].value, y_shape], stddev=0.1), name="last_w")
last_b = tf.Variable(tf.constant(0.1, shape=[y_shape]), name="last_b")
y_ = tf.nn.xw_plus_b(fit_stack, last_w, last_b)

cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y_stack))
train_op = tf.train.AdamOptimizer().minimize(loss=cost)

model_pred = tf.argmax(y_, 1)
correct_pred = tf.equal(model_pred, tf.argmax(y_stack, 1))
acc = tf.reduce_mean(tf.cast(correct_pred, "float"))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        for d_x, d_y in generate_data(train_x, train_y_onehot):
            sess.run(train_op, feed_dict={x_stack: d_x, y_stack: d_y})
        if i % 20 == 0:
            losses, correct_preds, accuracy = sess.run([cost, correct_pred, acc], feed_dict={x_stack: train_x, y_stack: train_y_onehot})
            print("losses is {}, acc is {}".format(losses, accuracy))

losses is 0.6378161311149597, acc is 0.3333333432674408
losses is 0.34858831763267517, acc is 0.7182666659355164
losses is 0.31274452805519104, acc is 0.7983666658401489
losses is 0.30125391483306885, acc is 0.8094666600227356
losses is 0.2911825478076935, acc is 0.8147333264350891
losses is 0.28002288937568665, acc is 0.8197000026702881
losses is 0.2727765440940857, acc is 0.8237000107765198
losses is 0.2838453948497772, acc is 0.8209666609764099
losses is 0.28483638167381287, acc is 0.8266666531562805
losses is 0.28455671668052673, acc is 0.8291333317756653
losses is 0.2806958854198456, acc is 0.8327000141143799
losses is 0.2758401930332184, acc is 0.8343999981880188
losses is 0.2722272276878357, acc is 0.8353999853134155
losses is 0.26942726969718933, acc is 0.8366333246231079
losses is 0.2718792259693146, acc is 0.8375333547592163
losses is 0.2703576385974884, acc is 0.8382666707038879
losses is 0.2679765522480011, acc is 0.8400333523750305
losses is 0.26488324999809265, acc is 0.8

# denoise AutoEncoder

In [84]:
hidden_size = 10
input_shape = 5

x = tf.placeholder(tf.float32, shape=[None, input_shape], name="input_x")
mask = tf.placeholder(tf.float32, shape=[None, input_shape], name="mask_s")

w_init_max = 4 * np.sqrt(6./ (hidden_size + input_shape))
w1 = tf.Variable(tf.random_uniform(shape=[input_shape, hidden_size], minval=-w_init_max,
                                  maxval=w_init_max), name="input_w1")
b1 = tf.Variable(tf.zeros([hidden_size]), name="b1")

w1_t = tf.transpose(w1)
b1_t = tf.Variable(tf.zeros([input_shape]), name="b1_t")

In [85]:
tild_x = tf.multiply(x, mask)
# 这里有主要 如何 使用masking
y = tf.nn.relu(tf.matmul(tild_x, w1) + b1)
z = tf.nn.sigmoid(tf.matmul(y, w1_t) + b1_t)
cost = tf.reduce_mean(tf.pow(z - x, 2))
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)

In [127]:
train_x = np.r_[np.random.randn(10000, 5), np.random.randn(10000, 5) + 2]
# train_y = np.r_[np.repeat(-1, 10000), np.repeat(1, 10000)].reshape(-1, 1)

def gen_data(x, size=50):
    alls = x.shape[0]
    n = int(alls / size)
    for i in range(n-1):
        yield x[size * i : size * (i+1)]
        

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        for d_x in gen_data(train_x):
            mask_vale = np.random.binomial(1, 0.8, size=d_x.shape)
            sess.run(train_op, feed_dict={x: d_x, mask: mask_vale})
        if i % 5==0:
            mask_vale = np.random.binomial(1, 0.8, size=train_x.shape)
            losses = sess.run(cost, feed_dict={x:train_x, mask: mask_vale})
            print("losses is {}".format(losses))
    print(sess.run(w1))
    print(sess.run(w1_t))
    mask_encoder = np.random.binomial(1, 1, size=train_x.shape)
    #encoder_x = sess.run(y, feed_dict={x: train_x, mask: mask_encoder})
    encoder_x = y.eval({x: train_x, mask: mask_encoder})
    print(encoder_x)

losses is 1.8198922872543335
losses is 1.4293465614318848
losses is 1.4079359769821167
losses is 1.3945034742355347
losses is 1.3875428438186646
losses is 1.3836427927017212
losses is 1.3799068927764893
losses is 1.3764300346374512
losses is 1.3767027854919434
losses is 1.374959111213684
[[-1.9464875  -1.109545    1.4265246   0.8759942  -2.2511616   1.343049
   0.23419555 -2.150114   -0.7549587   2.4260287 ]
 [-1.426449    1.1320349  -0.39795464 -1.8483213   0.83186305 -0.10572219
   1.0274603  -1.8152014  -0.9173398   0.99109113]
 [ 0.7949902  -1.8315792   0.31412092 -1.926579   -0.31639603 -1.1137276
   1.6101949  -0.99653935  1.1029608   0.79431856]
 [ 1.1451286  -1.9548414  -1.7246299  -0.05418967 -0.4079116  -0.22205326
   0.7431589  -0.15360783 -2.3336692   1.3408062 ]
 [-1.3907019  -0.99031746  0.09212247 -1.2605349  -0.1283643   1.6878622
   2.027463    1.3425846  -1.3327801  -0.20025294]]
[[-1.9464875  -1.426449    0.7949902   1.1451286  -1.3907019 ]
 [-1.109545    1.1320349  

In [136]:
class denoise_autoencoder:
    def __init__(self, hidden_size, input_shape):
        self.hidden_size = hidden_size
        self.input_shape = input_shape
        
        self.x = tf.placeholder(tf.float32, shape=[None, self.input_shape], name="input_x")
        self.mask = tf.placeholder(tf.float32, shape=[None, self.input_shape], name="mask_s")
        
        w_init_max = 4 * np.sqrt(6./ (self.hidden_size + self.input_shape))
        self.w1 = tf.Variable(tf.random_uniform(shape=[self.input_shape, self.hidden_size], minval=-w_init_max,
                                  maxval=w_init_max), name="input_w1")
        self._b1 = tf.Variable(tf.zeros([self.hidden_size]), name="b1")
        self._w1_t = tf.transpose(self.w1)
        self._b1_t = tf.Variable(tf.zeros([self.input_shape]), name="b1_t")
        self.cost, self.train_op, self.encoders = self.build_model()
        
    def build_model(self):
        tild_x = tf.multiply(self.x, self.mask)
        # 这里有主要 如何 使用masking
        y = tf.nn.relu(tf.matmul(tild_x, self.w1) + self._b1)
        z = tf.nn.sigmoid(tf.matmul(y, self._w1_t) + self._b1_t)
        cost = tf.reduce_mean(tf.pow(z - self.x, 2))
        train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
        return cost, train_op, y

            
with tf.Graph().as_default():
    session_conf = tf.ConfigProto(
    allow_soft_placement=True,
    log_device_placement=False)
    sess = tf.Session(config=session_conf)
    with sess.as_default():
        model = denoise_autoencoder(hidden_size=10, input_shape=5)
        sess.run(tf.global_variables_initializer())
        for i in range(10):
            for d_x in gen_data(train_x):
                mask_value = np.random.binomial(1, 0.8, size=d_x.shape)
                sess.run(model.train_op, feed_dict={model.x: d_x, model.mask: mask_value})
            if i % 5==0:
                mask_vale = np.random.binomial(1, 0.8, size=train_x.shape)
                losses = sess.run(model.cost, feed_dict={model.x:train_x, model.mask: mask_vale})
                print("losses is {}".format(losses))
        print(sess.run(model.w1))
        # print(sess.run(model.w1_t))
        mask_encoder = np.random.binomial(1, 1, size=train_x.shape)
        encoder_x = sess.run(model.encoders, feed_dict={model.x: train_x, model.mask: mask_encoder})
        #encoder_x = y.eval({x: train_x, mask: mask_encoder})
        print(encoder_x)

losses is 1.5699390172958374
losses is 1.4798840284347534
[[-0.03909982  0.78273857  1.8065187  -0.48890954  1.9585708   0.12119851
  -2.050288   -0.18503776 -1.1932139   0.55353874]
 [ 2.2645807  -0.30339217  1.4423703  -0.5439672  -1.3680559   1.2672769
  -0.9956454  -0.80652535  1.4804232  -2.2291338 ]
 [-0.26816654 -1.4777379   2.1162074   1.3133695   0.40296686 -2.2469344
   0.37932917 -0.86401117  1.1084111   1.1243018 ]
 [ 0.2811688  -1.7740858   1.2804466  -1.2298981   0.8681259  -0.2088778
  -0.95917493  2.3373055  -0.14740819 -0.6989715 ]
 [ 2.109717   -0.6849237   0.71669406  1.828332   -1.3892382  -0.2631936
  -2.0900462   0.89236265 -0.14709139  0.18654285]]
[[ 0.          0.          0.         ...  0.10422404  1.2223064
   1.3990352 ]
 [ 1.3104633   0.          5.4927497  ...  0.          2.2232661
   0.89440036]
 [ 1.2108825   0.          0.         ...  2.0686102   1.0236765
   0.        ]
 ...
 [ 7.8629255   0.          8.645921   ...  8.125151    0.
   2.5048468 ]
 [

In [91]:
# 10 表示 进行10次试验， 成功的概率为 0.2, 进行分析的
# np.random.binomial(10, 0.2, size=(10, 2))
# np.random.binomial(1, 1, size=(10, 2))